#Considerando o ano de 2019, quais foram os 5 partidos que mais gastaram (valor líquido) com serviço de táxi e que tiveram pelo menos uma restituição nesse período? (Apresentar ambos os valores limitados a duas casas decimais).

In [1]:
#importando sparksession
from pyspark.sql import SparkSession

#criando um objeto sparksession object e um appName 
spark = sparkSession=SparkSession.builder.appName("Pergunta4").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/01/16 22:05:30 WARN Utils: Your hostname, DESKTOP-6FAV8P8 resolves to a loopback address: 127.0.1.1; using 172.23.38.96 instead (on interface eth0)
24/01/16 22:05:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/16 22:05:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/16 22:05:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/16 22:05:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/01/16 22:05:32 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
# Fazendo a leitura do arquivo food_coded.csv
#df = spark.read.option("header", "true").csv("/Havan_Labs/Dados/cota-parlamentar.csv")

#Leitura de arquivo CSV
df= spark.read.format("csv").option("header", "true").load("/mnt/c/Havan_Labs/Dados/cota-parlamentar.csv")
df.show()

24/01/16 22:05:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+-------------------+------------+-----------+----------------+---------------------+------------+-------------+------+-----------------------+-------+------+----------+----------------+----------+---------+----+-----------------+--------------+--------------------+-------------------------+--------------------+---------+-------------+---------+------------+--------+----------+--------------+
|codlegislatura|         datemissao|idedocumento|idecadastro|indtipodocumento|nucarteiraparlamentar|nudeputadoid|nulegislatura|numano|numespecificacaosubcota|numlote|nummes|numparcela|numressarcimento|numsubcota|sgpartido|sguf|txnomeparlamentar|    txtcnpjcpf|        txtdescricao|txtdescricaoespecificacao|       txtfornecedor|txtnumero|txtpassageiro|txttrecho|vlrdocumento|vlrglosa|vlrliquido|vlrrestituicao|
+--------------+-------------------+------------+-----------+----------------+---------------------+------------+-------------+------+-----------------------+-------+------+-----

In [3]:
# utilizando funcoes do pyspark para filtro
from pyspark.sql.functions import *
# Filtrando os dados que nao sao nulos
resultdf_taxi = df.filter( (df.sgpartido.isNotNull()) & (df.vlrliquido.isNotNull() & (df.numano == 2019) )).select('numano'
                                                        ,'sgpartido'
                                                        ,'txtfornecedor'
                                                        ,'vlrdocumento'
                                                        ,'vlrglosa'
                                                        ,'vlrliquido'
                                                        ,'vlrrestituicao')

# Adicionando uma coluna indicando se a palavra "TAXI" está presente nos campos específicos
resultdf_taxi_com_taxi = resultdf_taxi.withColumn("contem_taxi",  
                                                   (instr(upper(col("txtfornecedor")), "TAXI") > 0) | 
                                                   (instr(upper(col("vlrdocumento")), "TAXI") > 0) | 
                                                   (instr(upper(col("vlrglosa")), "TAXI") > 0) | 
                                                   (instr(upper(col("vlrliquido")), "TAXI") > 0) | 
                                                   (instr(upper(col("vlrrestituicao")), "TAXI") > 0))

# Filtrando apenas os registros que contêm a palavra "TAXI"
resultdf_taxi_com_taxi_2 = resultdf_taxi_com_taxi.filter(col("contem_taxi"))

# Exibindo o resultado
resultdf_taxi_com_taxi_2.show()


+------+---------+--------------------+------------+--------+----------+--------------+-----------+
|numano|sgpartido|       txtfornecedor|vlrdocumento|vlrglosa|vlrliquido|vlrrestituicao|contem_taxi|
+------+---------+--------------------+------------+--------+----------+--------------+-----------+
|  2019|      PSC|           SINPETAXI|          30|       0|        30|          NULL|       true|
|  2019|       PT|ASSOCIACAO DOS MO...|          30|       0|        30|          NULL|       true|
|  2019|       PT|ASSOCIACAO DOS MO...|          30|       0|        30|          NULL|       true|
|  2019|       PT|ASSOCIACAO DOS MO...|          44|       0|        44|          NULL|       true|
|  2019|       PT|ASSOCIACAO DOS MO...|          44|       0|        44|          NULL|       true|
|  2019|       PT|ASSOCIACAO DOS MO...|          44|       0|        44|          NULL|       true|
|  2019|       PT|ASSOCIACAO DOS MO...|          44|       0|        44|          NULL|       true|


In [4]:
resultdf_taxi_com_taxi_3 = resultdf_taxi_com_taxi_2.withColumn('vlrdocumento_float', col("vlrdocumento").cast("float"))
resultdf_taxi_com_taxi_4 = resultdf_taxi_com_taxi_3.withColumn('vlrglosa_float', col("vlrglosa").cast("float"))
resultdf_taxi_com_taxi_5 = resultdf_taxi_com_taxi_4.withColumn('vlrliquido_float', col("vlrliquido").cast("float"))
resultdf_taxi_com_taxi_6 = resultdf_taxi_com_taxi_5.withColumn('vlrrestituicao_float', col("vlrrestituicao").cast("float"))

resultdf_taxi_com_taxi_6.show()

+------+---------+--------------------+------------+--------+----------+--------------+-----------+------------------+--------------+----------------+--------------------+
|numano|sgpartido|       txtfornecedor|vlrdocumento|vlrglosa|vlrliquido|vlrrestituicao|contem_taxi|vlrdocumento_float|vlrglosa_float|vlrliquido_float|vlrrestituicao_float|
+------+---------+--------------------+------------+--------+----------+--------------+-----------+------------------+--------------+----------------+--------------------+
|  2019|      PSC|           SINPETAXI|          30|       0|        30|          NULL|       true|              30.0|           0.0|            30.0|                NULL|
|  2019|       PT|ASSOCIACAO DOS MO...|          30|       0|        30|          NULL|       true|              30.0|           0.0|            30.0|                NULL|
|  2019|       PT|ASSOCIACAO DOS MO...|          30|       0|        30|          NULL|       true|              30.0|           0.0|       

In [5]:
# Criando uma tabela temporária em memória com os dados e utilizando consulta SQL
resultdf_taxi_com_taxi_6.createOrReplaceTempView("tab_cota_pergunta_4_taxi")

In [6]:
from pyspark.sql.window import Window

# utilizando funcoes do pyspark para filtro
from pyspark.sql.functions import *
# Filtrando os dados que nao sao nulos
resultdf_restituicao = df.filter((df.sgpartido.isNotNull()) & (df.vlrrestituicao.isNotNull() & (df.numano == 2019) )).select('numano','sgpartido','vlrrestituicao')

resultdf_restituicao_2 = resultdf_restituicao.withColumn('vlrrestituicao_float', col("vlrrestituicao").cast("float"))

# Criando uma janela para a soma por grupo
window_spec = Window.partitionBy("numano", "sgpartido")

# Adicionando uma nova coluna com a soma dos valores por grupo
resultdf_restituicao_3 = resultdf_restituicao_2.withColumn('soma_vlrrestituicao', sum('vlrrestituicao_float').over(window_spec))

# Removendo as colunas desnecessárias
resultdf_restituicao_4 = resultdf_restituicao_3.drop('vlrrestituicao', 'vlrrestituicao_float')

# Removendo duplicatas, caso existam
resultdf_restituicao_5 = resultdf_restituicao_4.dropDuplicates()

resultdf_restituicao_5.show()

+------+------------+-------------------+
|numano|   sgpartido|soma_vlrrestituicao|
+------+------------+-------------------+
|  2019|         MDB| 1300.7500247955322|
|  2019|        NOVO|  341.9600028991699|
|  2019|       PATRI|  9.029999732971191|
|  2019|       PCdoB|               15.0|
|  2019|         PDT| 1414.9100017547607|
|  2019|          PL|  731.2600004673004|
|  2019|          PP|  9816.049863815308|
|  2019|         PPS|  6.800000190734863|
|  2019|          PR|  20.34000015258789|
|  2019|        PROS|              170.0|
|  2019|         PSB| 300.73999786376953|
|  2019|         PSC|  231.1599998474121|
|  2019|         PSD| 20275.859972953796|
|  2019|        PSDB|    16542.580078125|
|  2019|         PSL|   9876.58001089096|
|  2019|          PT|  4440.750100135803|
|  2019|         PTB| 45.369998931884766|
|  2019|REPUBLICANOS|             8000.0|
+------+------------+-------------------+



In [7]:
# Criando uma tabela temporária em memória com os dados e utilizando consulta SQL
resultdf_restituicao_5.createOrReplaceTempView("tab_cota_pergunta_4_restituicao_group")

In [8]:

sparkSession.sql("select * from tab_cota_pergunta_4_restituicao_group  ").show()

+------+------------+-------------------+
|numano|   sgpartido|soma_vlrrestituicao|
+------+------------+-------------------+
|  2019|         MDB| 1300.7500247955322|
|  2019|        NOVO|  341.9600028991699|
|  2019|       PATRI|  9.029999732971191|
|  2019|       PCdoB|               15.0|
|  2019|         PDT| 1414.9100017547607|
|  2019|          PL|  731.2600004673004|
|  2019|          PP|  9816.049863815308|
|  2019|         PPS|  6.800000190734863|
|  2019|          PR|  20.34000015258789|
|  2019|        PROS|              170.0|
|  2019|         PSB| 300.73999786376953|
|  2019|         PSC|  231.1599998474121|
|  2019|         PSD| 20275.859972953796|
|  2019|        PSDB|    16542.580078125|
|  2019|         PSL|   9876.58001089096|
|  2019|          PT|  4440.750100135803|
|  2019|         PTB| 45.369998931884766|
|  2019|REPUBLICANOS|             8000.0|
+------+------------+-------------------+



In [9]:

sparkSession.sql("select * from tab_cota_pergunta_4_taxi  ").show()

+------+---------+--------------------+------------+--------+----------+--------------+-----------+------------------+--------------+----------------+--------------------+
|numano|sgpartido|       txtfornecedor|vlrdocumento|vlrglosa|vlrliquido|vlrrestituicao|contem_taxi|vlrdocumento_float|vlrglosa_float|vlrliquido_float|vlrrestituicao_float|
+------+---------+--------------------+------------+--------+----------+--------------+-----------+------------------+--------------+----------------+--------------------+
|  2019|      PSC|           SINPETAXI|          30|       0|        30|          NULL|       true|              30.0|           0.0|            30.0|                NULL|
|  2019|       PT|ASSOCIACAO DOS MO...|          30|       0|        30|          NULL|       true|              30.0|           0.0|            30.0|                NULL|
|  2019|       PT|ASSOCIACAO DOS MO...|          30|       0|        30|          NULL|       true|              30.0|           0.0|       

In [10]:

group_df = sparkSession.sql("select taxi.numano , taxi.sgpartido, sum(taxi.vlrliquido_float) soma_vlrliquido_float  \
                 from tab_cota_pergunta_4_taxi taxi \
                 group by taxi.numano , taxi.sgpartido \
                 order by sum(taxi.vlrliquido_float) desc \
                 ")

In [11]:
# Criando uma tabela temporária em memória com os dados e utilizando consulta SQL
group_df.createOrReplaceTempView("tab_cota_pergunta_4_taxi_group")

In [12]:
sparkSession.sql("select * from tab_cota_pergunta_4_taxi_group \
                 ").show()

+------+-------------+---------------------+
|numano|    sgpartido|soma_vlrliquido_float|
+------+-------------+---------------------+
|  2019|           PP|   253821.89998531342|
|  2019|          PSD|   240785.07000160217|
|  2019|          PTB|             152532.0|
|  2019|         PSDB|   132750.82000350952|
|  2019|           PL|   101394.49000549316|
|  2019|          DEM|    99233.81999588013|
|  2019|          PDT|    85158.21999549866|
|  2019|          PSB|    74623.80000305176|
|  2019|           PT|    68422.15998268127|
|  2019| REPUBLICANOS|    51057.21000003815|
|  2019|          MDB|    38277.40000152588|
|  2019|          PRB|              34125.0|
|  2019|SOLIDARIEDADE|    33340.37999916077|
|  2019|          PSL|    24598.86000442505|
|  2019|         PODE|    21497.19000530243|
|  2019|          PSC|   19346.180000305176|
|  2019|          PPS|               9050.0|
|  2019|         PSOL|    7578.660026550293|
|  2019|         PROS|               6700.0|
|  2019|  

In [13]:

df_final = sparkSession.sql("select taxi.numano , taxi.sgpartido, taxi.soma_vlrliquido_float, rest.soma_vlrrestituicao  \
                 from tab_cota_pergunta_4_taxi_group taxi \
                 inner join tab_cota_pergunta_4_restituicao_group rest on rest.numano = taxi.numano and rest.sgpartido = taxi.sgpartido \
                 order by taxi.soma_vlrliquido_float desc \
                 ").limit(5)

df_final.show()

+------+---------+---------------------+-------------------+
|numano|sgpartido|soma_vlrliquido_float|soma_vlrrestituicao|
+------+---------+---------------------+-------------------+
|  2019|       PP|   253821.89998531342|  9816.049863815308|
|  2019|      PSD|   240785.07000160217| 20275.859972953796|
|  2019|      PTB|             152532.0| 45.369998931884766|
|  2019|     PSDB|   132750.82000350952|    16542.580078125|
|  2019|       PL|   101394.49000549316|  731.2600004673004|
+------+---------+---------------------+-------------------+

